# cutting tool for a worm gear

1. idea 1:  

<img src="../../docs/computing a profile_from_a_given_rack.jpg">

In [4]:
import sympy as sp
import numpy as np

In [5]:
def symbolic_transformation(angle, axis, translation=np.array([0., 0., 0.])):
    """
        see http://en.wikipedia.org/wiki/SO%284%29#The_Euler.E2.80.93Rodrigues_formula_for_3D_rotations
        sympy enabled transformation
        angle: angle of rotation
        axis: the axis of the rotation
        translation: translation of transformation
    """
    assert len(axis) == 3
    a = sp.cos(angle / 2)
    axis_normalized = axis / sp.sqrt(axis.dot(axis))
    (b, c, d) = -axis_normalized * sp.sin(angle / 2)
    mat = sp.Matrix(
        [
            [
                a**2 + b**2 - c**2 - d**2,
                2 * (b * c - a * d),
                2 * (b * d + a * c),
                translation[0],
            ],
            [
                2 * (b * c + a * d),
                a**2 + c**2 - b**2 - d**2,
                2 * (c * d - a * b),
                translation[1],
            ],
            [
                2 * (b * d - a * c),
                2 * (c * d + a * b),
                a**2 + d**2 - b**2 - c**2,
                translation[2],
            ],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )
    return sp.simplify(mat)


In [7]:
t = sp.Symbol("t")
T1 = symbolic_transformation(np.pi / 2.,
                             np.array([1., 0., 0.]),
                             np.array([12.5,0., 1.15]))
T2 = symbolic_transformation(-t / 7.5,
                             np.array([0., 0., 1.]),
                             np.array([0., 0., 0.]))
T3 = symbolic_transformation(0.,
                             np.array([1., 0., 0.]),
                             np.array([0., 0., t]))

T = sp.nsimplify(T2.inv() @ T1.inv() @ T3, tolerance=10e-16)

In [8]:
T_fn = sp.lambdify(t, T)
dT_fn = sp.lambdify(t, T.diff(t))
dT_fn(0.)

array([[ 0.        ,  0.        , -0.13333333,  0.15333333],
       [-0.13333333,  0.        ,  0.        ,  0.66666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

Diese Methode funktioniert nicht, weil die Bedingung zur Bestimmung des Kontaktpunktes falsch ist.
Eine Bedingung für die generierung einer konstanten Übersetzung ist, dass die Normale auf die Kontaktfläche (Zahnstange) immer durch den Punkt p (Eingriffspunkt für Ersatzzahnrad (Zylinder) und Ersatzzahnstange (Quader) gehen muss.
Gesucht sind also Punkte auf der Fläche S welche verbunden mit P normal auf die Fläche stehen. Dies kann auch als minimaler Abstand von P zur Fläche gesehen werden.

für jedes u: min(norm(S(u,v)-P)) -> d(norm(S(u,v)-P)/dv = 0
die Änderung des Abstands ist 0 -> die Gerade steht normal auf die Fläche


## Vorgehensweise

1. approximate the surface by a BSplineSurface

- Erstellen eines "Cross-Sektion" objekts aus dem "Werkzeug"

<img src="cross_section.png">  

- Draft downgrade um Kanten zu bekommen
das Cross-section Objekt beinhaltet nicht die anfangs und end Kanten. Diese müssen zusätzlich vom Werkzeug extrahiert werden

- Part Loft zum erstellen einer schönen BSplinefläche

<img src="loft_bspline_flaechen.png"> 

3. Loft -> Surface

```python
# select the cutting faces
face_1 = App.ActiveDocument.Loft.Shape.Faces[0].copy()
face_2 = App.ActiveDocument.Loft001.Shape.Faces[0].copy()

# compute the contact curve:
bsp_1 = face_1.Surface
bsp_2 = face_2.Surface

```

4. Minimierung des Abstands zum "Pitch-Punkt"

```python
import scipy as scp
point = App.Vector(5., 0., 1.15 - time) 
xyz_1 = []
for v in np.linspace(0, 1, 5):
        def dist_1(u):
            distance = bsp_1.value(u, v) - point
            return distance.x ** 2 + distance.z ** 2
        u_1 = scp.optimize.minimize(dist_1, 0.5, tol=1e-6).x[0]
        xyz_1.append(bsp_1.value(u_1, v))
```

5. erstellen einer B-Spline Kurve welche durch die Kinematik T transformiert wird

```python
c_1 = part.BSplineCurve()
c_1.interpolate(Points=xyz_1)
c_1 = c_1.toShape()

part.show(c_1.transformShape(T))
```

6. Loft anwenden auf die erstellten BSpline Kurven

<img src="loft_of_generated_bsplines.png">

7. Array für das Zahnrad

<img src="gear_assembly.png">

In [6]:
import sympy as sp
t, x, z = sp.symbols(["t", "x", "z"], real=True)
s, alpha, n_t, y = sp.symbols(["s", "alpha", "n_t", "y"], real=True, positiv=True)

In [7]:
T_spiral = symbolic_transformation(t * n_t / sp.pi, np.array([0, 0, 1]), np.array([0, 0, t * n_t]))
T_spiral

Matrix([
[ cos(n_t*t/pi), sin(n_t*t/pi), 0,     0],
[-sin(n_t*t/pi), cos(n_t*t/pi), 0,     0],
[             0,             0, 1, n_t*t],
[             0,             0, 0,   1.0]])

In [22]:
l = sp.Matrix([0, s * sp.cos(alpha), s * sp.sin(alpha), 1])
l

Matrix([
[           0],
[s*cos(alpha)],
[s*sin(alpha)],
[           1]])

In [23]:
spiral = (T_spiral @ l)
spiral

Matrix([
[s*sin(n_t*t/pi)*cos(alpha)],
[s*cos(alpha)*cos(n_t*t/pi)],
[      n_t*t + s*sin(alpha)],
[                       1.0]])

In [32]:
x_cross_section = sp.simplify(sp.solve(spiral[0] - x, t)[1])
x_cross_section  # parameter s

pi*asin(x/(s*cos(alpha)))/n_t

In [33]:
spiral_x = sp.simplify(spiral.subs({t: x_cross_section}))
spiral_x

Matrix([
[                                               x],
[s*sqrt(1 - x**2/(s**2*cos(alpha)**2))*cos(alpha)],
[        s*sin(alpha) + pi*asin(x/(s*cos(alpha)))],
[                                             1.0]])

In [43]:
y_cross_section = sp.simplify(sp.solve(spiral_x[1]- y, s)[0])
y_cross_section

sqrt(x**2 + y**2)/cos(alpha)

In [44]:
spiral_xy = sp.simplify(spiral_x.subs({s: y_cross_section}))
spiral_xy

Matrix([
[                                                          x],
[                                                     Abs(y)],
[sqrt(x**2 + y**2)*tan(alpha) + pi*asin(x/sqrt(x**2 + y**2))],
[                                                        1.0]])